In [1]:
import requests
from collections import defaultdict
import pandas as pd
import numpy as np

##  APIs

1. Forest: AG.LND.FRST.ZS
2. Permanent cropland: AG.LND.CROP.ZS
3. Agricultural Land: AG.LND.AGRI.ZS
4. Arable Land: AG.LND.ARBL.ZS
5. Land Cereal production : AG.LND.CREL.HA
6. Rural population: SP.RUR.TOTL.ZS
7. Urban Population:SP.URB.TOTL.IN.ZS
8. Male Employment:SL.AGR.EMPL.MA.ZS
9. Female Employement: SL.AGR.EMPL.FE.ZS
10. Fertilizer Use: AG.CON.FERT.ZS
11. Cereal Production: AG.YLD.CREL.KG
12. % of GDP value: NV.AGR.TOTL.ZS
13. Total Population: SP.POP.TOTL
14. Under 5 mortality: SH.DYN.MORT
15. Greenhouse Gases: EN.ATM.GHGT.KT.CE
16. CO2 emissions: EN.ATM.CO2E.KT
17. Poverty : SI.POV.DDAY

In [2]:
country_list = ['World', 'Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong SAR, China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kosovo', 'Kuwait', 'Kyrgyz Republic', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico', 'Micronesia, Fed. Sts.', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovak Republic', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'St. Kitts and Nevis', 'St. Lucia', 'St. Martin (French part)', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkiye', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, RB', 'Vietnam', 'Virgin Islands (U.S.)', 'West Bank and Gaza', 'Yemen, Rep.', 'Zambia', 'Zimbabwe']

In [3]:
%store largest_10_pop_df
%store countries_population_df

UsageError: Unknown variable 'largest_10_pop_df'


In [4]:
def removeFormatColumns(df):
    """
    _summary_

    Args:
        df (DataFrame): _description_

    Returns:
        DataFrame: _description_
    """
    df.drop(columns=['indicator','obs_status','decimal','countryiso3code', 'unit'], inplace=True, axis=1)

    return df

def formatColumns(df):
    """
    _summary_

    Args:
        df (DataFrame): _description_
    Returns:
        df (DateFrame):
    """
    
    df["date"] = pd.to_datetime(df["date"])

    #turn country feature into just country name
    for i, country in enumerate(df['country']):
        df.loc[i,'country'] = country['value']

    return df

def justCountries(df):
    """
    _summary_
    Args:
        df (DataFrame): _description_
    Returns:
        DataFrame: _description_
    """
    df=df[df['country'].isin(country_list)]
    return df

In [28]:
# list of worldbank API indicators that will be used to form a database
world_bank_api = ['SP.POP.TOTL', 'AG.LND.TOTL.K2', 'AG.LND.FRST.ZS',
'AG.LND.CROP.ZS','AG.LND.AGRI.ZS','AG.LND.ARBL.ZS','AG.LND.CREL.HA', 
'SP.RUR.TOTL.ZS','SP.URB.TOTL.IN.ZS', 'SL.AGR.EMPL.MA.ZS', 'SL.AGR.EMPL.FE.ZS', 'AG.CON.FERT.ZS','AG.YLD.CREL.KG', 'NV.AGR.TOTL.ZS', 'SH.DYN.MORT', 
'EN.ATM.GHGT.KT.CE', 'EN.ATM.CO2E.KT', 'SI.POV.DDAY']

# loop to create a list of URLs from api indicators
urls = []
for indicator in world_bank_api:
    url = 'http://api.worldbank.org/v2/countries/indicators/' + indicator 
    urls.append(url)
dataframe_list = []

# loop to get request each url and iterate through 18 pages of json data, then turn into a list of dataframes.

for url in urls:
    data = []
    try:  
        for page in range(1,18):
            payload = {'format': 'json', 'per_page': '1000', 'date':'1960:2022', 'page':page}     
            r = requests.get(url, params=payload)
            data+=r.json()[1]
        
        dataframe_list.append(pd.DataFrame(data))

    except:
        print('could not load data', url)

In [29]:
world_bank_columns = ['population', 'total_land_sqkm', 'forest_%','crop_%','agricultural_%','arable_%','cereal_grain_hectare', 'rural_pop_%','urban_pop_%', 'male_employement_ag', 'female_employment_ag', 'fertilizer_consump','cereal_yield_hectare', 'total_gdp_ag_forestry_fishing', 'mortality_under5', 'Total_Greenhouse_gases', 'CO2_emmission','Poverty_under1_90_per_day']

world_bank_df = None

#format and combine datframes into a single dataframe
for i, df in enumerate(dataframe_list):
  df = removeFormatColumns(df)
  df = formatColumns(df)
  df = justCountries(df)
  
  if world_bank_df is not None:
    world_bank_df.insert(loc=len(world_bank_df.columns),column=world_bank_columns[i], 
    value=df['value'])
  else:
    world_bank_df = pd.DataFrame(df)
    world_bank_df.rename(columns={'value' : world_bank_columns[i]}, inplace=True)

In [30]:
# turning % into total features

world_bank_df['total_urban_pop'] = world_bank_df['urban_pop_%']*world_bank_df['population'] / 100

world_bank_df['total_rural_pop'] = world_bank_df['rural_pop_%']*world_bank_df['population'] / 100

world_bank_df['total_forest_land'] = world_bank_df['forest_%']*world_bank_df['total_land_sqkm'] / 100

world_bank_df['total_ag_land'] = world_bank_df['agricultural_%']*world_bank_df['total_land_sqkm'] / 100

world_bank_df['total_perm_crop_land'] = world_bank_df['crop_%']*world_bank_df['total_land_sqkm'] / 100

world_bank_df['total_arable_land'] = world_bank_df['arable_%']*world_bank_df['total_land_sqkm'] / 100


In [31]:
#countries with the top 10 populations
largest_10_population = world_bank_df[world_bank_df['date']=='2021'].nlargest(n=11,columns='population')

largest_10_list = [country for country in largest_10_population['country']]

largest_10_pop_df = world_bank_df[world_bank_df['country'].isin(largest_10_list)]

In [32]:
%store world_bank_df
%store largest_10_pop_df

Stored 'world_bank_df' (DataFrame)
Stored 'largest_10_pop_df' (DataFrame)
